In [1]:
import zstd
import os
import sys
import json
import time
import warnings
import numpy as np
from tqdm import tqdm
from random import shuffle
import heapq
import hlt

In [2]:
warnings.filterwarnings('ignore')

In [3]:
replay_dir = '../../../Replays/'

In [29]:
class CyclicalList:
    """
    ReplayManager
    """
    
    


    def __init__(self, std_list):
        self.idx = 0
        self.len = len(std_list)
        self.list = std_list
        # Init constants
        self.SHIPS_PER_TEAM = 30.0
        self.FEATURES_PER_SHIP = 6.0
        self.MAX_PLANETS = 14.0
        self.MIN_PLANETS = 12.0
        self.AVG_TOTAL_PLANETS = 13.0
        self.MAX_SHIP_HEALTH = 255.0
        self.MAX_VELOCITY = 7.0
        self.MAX_MAP_WIDTH = 384.0 
        self.MAX_MAP_HEIGHT = 256.0 
        self.MAX_PLANETS_SECTION = 4.0
        self.FEATURES_PER_PLANET = 5.0
        self.MAX_OBJECT_DISTANCE = 461.510563
        self.APPROXIMATE_AVG_PLANET_HEALTH = 2022.0
        self.MAX_PLANET_RADIUS = 16.0
        
    def getNext(self, shuffle = False):
        self.idx = (self.idx + 1) % self.len
        if shuffle and self.idx == 0:
            shuffle(self.list)
        return self.list[self.idx]
    
    def getReplaysWithSamples(min_samples):
        """
        Iterates the buffer and gets the next replays
        that have at least min_samples overall.
        """
        cur_samples = 0
        replays = []
        for i in range(self.len):
            file_name = self.getNext(shuffle = True)
            replay = self.getReplayContent(file_name)
            replays.append(replay)
            cur_samples += replay['num_frames']
            if(cur_samples > min_samples):
                break
        return replays
    
    #def sampleFrames(self, replay_list, )
    
    def getReplayContent(self, replay_dir, file_name):
        with open(replay_dir + file_name, "rb") as rpfile:    
            decoded_data = zstd.decompress((rpfile.read())).decode('utf-8')
            replay_json = json.loads(decoded_data.strip())
            return replay_json
        
    def createFeatureVector(self, frame, timestep, map_width, map_height, planets, ship_id):
        # Pre-allocate matrix  
        feature_vector = np.array((17 + self.SHIPS_PER_TEAM * self.FEATURES_PER_SHIP * 2 \
                                   + self.MAX_PLANETS_SECTION * self.FEATURES_PER_PLANET * 3 , 1))
        
        # Store team sizes
        our_team_size, their_team_size = self.getTeamSize(frame, ship_id)
        feature_vector[0] = our_team_size
        feature_vector[1] = float(our_team_size) / their_team_size
        
        # Store percentage of planets per team
        total_planets_norm, planet_balance, neutral_fraction = self.getPlanetAggregatedFeatures(frame, ship_id)
        feature_vector[2] = total_planets_norm
        feature_vector[3] = planet_balance
        feature_vector[4] = neutral_fraction
        
        # Store timestep
        feature_vector[5] = timestep
        
        # Store our ship features
        health_norm, docking_one_hot, dist_hor_norm, dist_vert_norm, one_hot_x, one_hot_y = \
                                                                self.getOurShipFeatures(frame, ship_id, map_width, map_height)
        feature_vector[6] = health_norm
        feature_vector[7] = docking_one_hot
        feature_vector[8:11] = docking_one_hot
        feature_vector[11] = dist_hor_norm
        feature_vector[12] = dist_vert_norm
        feature_vector[13:15] = one_hot_x
        feature_vector[15:17] = one_hot_y
        
        # Store ship features
        ship_features = self.getShipFeatures(frame, ship_id)
        ships_start = 17
        ships_end = ships_start + self.SHIPS_PER_TEAM * self.FEATURES_PER_SHIP * 2
        feature_vector[ships_start:ships_end] = ship_features
        #feature_vector = np.vstack((feature_vector, ship_features))
        
        # Store planet features
        planet_features = self.getPlanetFeatures(frame, ship_id, planets)
        planet_start = ships_end
        feature_vector[planet_start:] = planet_features
        
        return feature_vector
    
    def getPlanetFeatures(self, frame, our_ship_id, planets):
        """
        Get particular planets' features by sections
        Args:
            ...
        Returns:
            ...
        """
        ships = frame['ships']
        our_team_id = self.getShipOwner(ships, our_ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        our_ship_info = ships[our_team_id][our_ship_id]

        # Get list of ally and enemy ships
        our_planets = []
        enemy_planets = []
        neutral_planets = []
        for planet in planets:
            planetId = str(planet['id'])
            planetObj = frame['planets'][planetId]
            # Get difference in position coordinates
            diff_y = planet['y'] - our_ship_info['y']
            diff_x = planet['x'] - our_ship_info['x']
            
            # Calculate and normalize distance
            planetDist = np.sqrt(diff_x ** 2 + diff_y **2)
            normalized_planetDist = (planetDist - MAX_OBJECT_DISTANCE / 2) / MAX_OBJECT_DISTANCE
   
            # Calculate relative angle difference between ship and planet
            angle_diff = np.arctan2(diff_y, diff_x)    
            angle_diff_norm = angle_diff / (2 * np.pi) # Normalize angle difference to [-0.5, 0.5]
            
            # Normalize radius
            norm_radius = (planet['r'] - self.MAX_PLANET_RADIUS / 2) / self.MAX_PLANET_RADIUS 
            
            # Approximately normalize health
            norm_health = (planetObj['health'] - self.APPROXIMATE_AVG_PLANET_HEALTH) / ( 2 * APPROXIMATE_AVG_PLANET_HEALTH)
            
            # Get number of docked ships normalized
            docked_ships = len(planetObj['docked_ships'])
            max_spots = planet['docking_spots']
            norm_docked_ships = (docked_ships - max_spots / 2) / max_spots
            
            # Create list with attributes
            planetAttr = (normalized_planetDist, norm_health, angle_diff_norm, norm_radius, norm_docked_ships)
            
            if planetObj['owner'] is None:
                neutral_planets.append(planetAttr)
            elif str(planetObj['owner']) == our_team_id:
                our_planets.append(planetAttr)
            else:
                enemy_planets.append(planetAttr)
                
 
        # Sort list of ally/enemy planets based on distance from us
        neutral_planets = sorted(neutral_planets, key = lambda x: x[0])
        our_planets = sorted(our_planets, key = lambda x: x[0])
        enemy_planets = sorted(enemy_planets, key = lambda x: x[0])

        # Preallocate memory for planet features
        planet_features = np.zeros((self.MAX_PLANETS_SECTION * self.FEATURES_PER_PLANET * 3, 1))
        
        # Get features for neutral planets
        for i in range(min(len(neutral_planets), self.MAX_PLANETS_SECTION)):
            # Save planet features
            start = i * self.FEATURES_PER_PLANET
            end = start + self.FEATURES_PER_PLANET
            planet_features[start:end] = np.array(neutral_planets[i])

        # Get features for neutral planets
        for i in range(min(len(our_planets), self.MAX_PLANETS_SECTION)):
            # Save planet features
            start = self.FEATURES_PER_PLANET * self.MAX_PLANETS_SECTION + i * self.FEATURES_PER_PLANET
            end = start + self.FEATURES_PER_PLANET
            planet_features[start:end] = np.array(our_planets[i])

        # Get features for neutral planets
        for i in range(min(len(enemy_planets), self.MAX_PLANETS_SECTION)):
            # Save planet features
            start = self.FEATURES_PER_PLANET * self.MAX_PLANETS_SECTION * 2 + i * self.FEATURES_PER_PLANET
            end = start + self.FEATURES_PER_PLANET
            planet_features[start:end] = np.array(enemy_planets[i])             
        
        return planet_features
                
                
    def getShipFeatures(self, frame, our_ship_id):
        """
        Get features of all the ships
        """
        ships = frame['ships']
        our_team_id = self.getShipOwner(ships, our_ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        our_ship_info = ships[our_team_id][our_ship_id]
        
        # Get list of ally and enemy ships
        ally_ships_list = list(ships[our_team_id].values())
        enemy_ships_list = list(ships[their_team_id].values())
        # Get distance between us and ally/enemy ships
        ally_ship_distances = [self.getShipDistance(ally_ship, our_ship_info) for ally_ship in ally_ships_list]
        enemy_ship_distances = [self.getShipDistance(enemy_ship, our_ship_info) for enemy_ship in enemy_ships_list]
           
        # Sort list of ally/enemy ships based on distance from us
        sorted_ally_lists = list(zip(*sorted(zip(ally_ship_distances,ally_ships_list), key=lambda ship: ship[0])))
        sorted_enemy_lists = list(zip(*sorted(zip(enemy_ship_distances,enemy_ships_list), key=lambda ship: ship[0])))
                    
        ship_features = np.zeros((self.SHIPS_PER_TEAMS * self.FEATURES_PER_SHIP * 2, 1))
        
        counter = 1
        # Get features for ally ships
        for i in range(len(ally_ships_list)):
            if (sorted_ally_lists[i][1]['id'] == our_ship_id):
                continue
            counter += 1
            
            # Get ship features
            distance = sorted_ally_lists[i][0]
            norm_distance = (distance - MAX_OBJECT_DISTANCE / 2) / MAX_OBJECT_DISTANCE # Normalize to [-0.5,0.5]
            angle_diff_norm, health_norm, docking_one_hot  = getRelativeShipFeatures(sorted_ally_lists[i][1], \
                                                                                                           our_ship_info)
            # Save features
            start = i * self.FEATURES_PER_SHIP
            end = start + self.FEATURES_PER_SHIP
            ship_features[start:end] = np.array([norm_distance, angle_diff_norm, health_norm, *docking_one_hot])
            
            if (counter >= self.SHIPS_PER_TEAMS):
                break

        # Get features for enemy ships
        for i in range(min(self.SHIPS_PER_TEAMS, len(enemy_ships_list))):
            # Get ship features
            distance = sorted_enemy_lists[i][0]
            norm_distance = (distance - MAX_OBJECT_DISTANCE / 2) / MAX_OBJECT_DISTANCE # Normalize to [-0.5,0.5]
            angle_diff_norm, health_norm, docking_one_hot  = getRelativeShipFeatures(sorted_enemy_lists[i][1], \
                                                                                                           our_ship_info)
            # Save features
            start = self.SHIPS_PER_TEAMS + i * self.FEATURES_PER_SHIP
            end = start + self.FEATURES_PER_SHIP
            ship_features[start:end] = np.array([norm_distance, angle_diff_norm, health_norm, *docking_one_hot])
        
        return ship_features
        
            
    def getRelativeShipFeatures(self, ship, our_ship):
        """
        Get features of a ship relative to our ship.
        
        Args:
            ship: Json object with ship
            our_ship: Json object with our ship
            our_angle: 
            
        Returns:
            Tuple of features extracted for ship.
        """        
        # Get difference in position coordinates
        diff_y = ship['y'] - our_ship['y']
        diff_x = ship['x'] - our_ship['x']
        
        # Calculate relative angle difference between ships
        angle_diff = np.arctan2(diff_y, diff_x)    
        angle_diff_norm = angle_diff / (2 * np.pi) # Normalize angle difference to [-0.5, 0.5]

        # Calculate and normalize ship health
        health_norm = (ship['health'] - MAX_SHIP_HEALTH / 2) / MAX_SHIP_HEALTH
        
        # Get docking status
        docking_one_hot =   [1, 0, 0] if ship['docking']['status'] == 'docked' else \
                            [0, 1, 0] if ship['docking']['status'] == 'undocked' else \
                            [0, 1, 1] if ship['docking']['status'] == 'docking' else \
                            [1, 0, 1]
        
        return angle_diff_norm, health_norm, docking_one_hot
        
    def getShipDistance(self, ship1, ship2):
        """
        Get distance between two ships
        
        Args:
            ship1: Json object for first ship
            ship2: Json object for second ship
            
        Returns:
            Euclidean distance between two ships
        """
        return np.sqrt((ship1['x'] - ship2['x']) ** 2 + (ship1['y'] - ship2['y']) ** 2)
        
        
    
    def getOurShipFeatures(self, frame, ship_id, map_width, map_height):
        """
        Get features for our own ship.
        """
        ships = frame['ships']
        our_team_id = self.getShipOwner(ships, ship_id)
        ship_info = ships[our_team_id][ship_id]
        
        # Get health 
        health_norm = (ship_info['health'] - MAX_SHIP_HEALTH / 2) / MAX_SHIP_HEALTH
        
       
        # Get docking status
        docking_one_hot =   [1, 0, 0] if ship_info['docking']['status'] == 'docked' else \
                            [0, 1, 0] if ship_info['docking']['status'] == 'undocked' else \
                            [0, 1, 1] if ship_info['docking']['status'] == 'docking' else \
                            [1, 0, 1]
        
        # Get information about angle and distance from closest borders
        dist_hor, dist_vert, one_hot_x, one_hot_y = self.getBorderFeatures(ship_info, map_height, map_width)
        # Normalize
        dist_hor_norm = (dist_hor - self.MAX_MAP_WIDTH / 4) / (self.MAX_MAP_WIDTH / 2)
        dist_vert_norm = (dist_vert - self.MAX_MAP_HEIGHT / 4) / (self.MAX_MAP_HEIGHT / 2)

        
        return health_norm, docking_one_hot, dist_hor_norm, dist_vert_norm, one_hot_x, one_hot_y
        
        
        
    def getBorderFeatures(self, ship_info, map_height, map_width):
        """
        Get distance and angle from closest horizontal and vertical border of the map
        """
        x_coord = ship_info['x']
        y_coord = ship_info['y']
        angle = np.arctan2(ship_info['vel_y'], ship_info['vel_x'])
        
        if x_coord < map_width / 2:
            # Left side
            dist_hor = x_coord
            one_hot_x = [1, 0]
        else:
            # Right side
            dist_hor = map_width-x_coord
            one_hot_x = [0, 1]
        
        if y_coord < map_height / 2:
            # Down
            dist_vert = y_coord
            one_hot_y = [1, 0]
        else:
            # Up
            dist_vert = map_height - y_coord
            one_hot_y = [0, 1]
        
        return dist_hor, dist_vert, one_hot_x, one_hot_y
    
    def getShipOwner(self, ships, ship_id):
        """
        Get index of ship owner
        """
        if ship_id in ships['0'].keys():
            our_team = '0'
        else:
            our_team = '1'
        return our_team
    
    def getTeamSize(self, frame, ship_id):
        """
        Get tuple of size for both teams
        """
        ships = frame['ships']
        our_team_id = self.getShipOwner(ships, ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        our_team_size = len(ships[our_team_id])
        their_team_size = len(ships['1'])
        return our_team_size, their_team_size
    
    def getPlanetPercentages(self, frame, ship_id):
        """
        Deprecated
        """
        our_team_id = self.getShipOwner(frame['ships'], ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        planets = frame['planets']
        total_planet_no = len(planets)
        owned_planets = np.sum([[1, 0] if str(planet['id']) == our_team_id else [0, 1] \
                               for planet in planets if planet['id'] is not None], axis = 0)
        our_planets_perc_norm = owned_planets[0] / total_planet_no - 0.5
        their_planets_perc_norm = owned_planets[1] / total_planet_no - 0.5
        return  our_planets_perc_norm, their_planets_perc_norm
        
        
    
    def getPlanetAggregatedFeatures(self, frame, ship_id):   
        """
        """
        # Get ids
        our_team_id = self.getShipOwner(frame['ships'], ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        
        # Get numbers of planets
        planets = frame['planets']
        total_planet_no = len(planets)
        owned_planets = np.sum([[1, 0] if str(planet['id']) == our_team_id else [0, 1] \
                               for planet in planets if planet['id'] is not None], axis = 0)
        our_planets = owned_planets[0]
        their_planets = owned_planets[1]
        neutral_planets = total_planet_no - (our_planet + their_planets)
        
        balance_normalized = (our_planets - their_planets) / (2 * total_planet_no)
        neutral_fraction = neutra_planets / total_planet_no - 0.5 # Subtract 0.5 to center at 0
        
        return (total_planet_no - self.AVG_TOTAL_PLANETS) / (self.MAX_PLANETS - self.MIN_PLANETS), \
                balance_normalized, neutral_fraction
        
 

In [6]:
replay_filename_list = os.listdir(replay_dir)
for file_name in os.listdir(replay_dir):
    if file_name.startswith('replay'):
        replay_filename_list.append(file_name)
replay_buffer = CyclicalList(replay_filename_list)

In [12]:
replay_json = replay_buffer.getReplayContent(replay_dir, replay_buffer.getNext())

In [27]:
replay_json['planets']

[{'docking_spots': 3,
  'health': 2022,
  'id': 0,
  'production': 1141,
  'r': 7.93006241065185,
  'x': 179.921908592795,
  'y': 123.921908592795},
 {'docking_spots': 3,
  'health': 2022,
  'id': 1,
  'production': 1141,
  'r': 7.93006241065185,
  'x': 156.078091407205,
  'y': 123.921908592795},
 {'docking_spots': 3,
  'health': 2022,
  'id': 2,
  'production': 1141,
  'r': 7.93006241065185,
  'x': 156.078091407205,
  'y': 100.078091407205},
 {'docking_spots': 3,
  'health': 2022,
  'id': 3,
  'production': 1141,
  'r': 7.93006241065185,
  'x': 179.921908592795,
  'y': 100.078091407205},
 {'docking_spots': 5,
  'health': 3788,
  'id': 4,
  'production': 2139,
  'r': 14.8557870626006,
  'x': 207.718541565376,
  'y': 50.6244515068243},
 {'docking_spots': 5,
  'health': 3788,
  'id': 5,
  'production': 2139,
  'r': 14.8557870626006,
  'x': 211.628401940932,
  'y': 167.875236439427},
 {'docking_spots': 5,
  'health': 3788,
  'id': 6,
  'production': 2139,
  'r': 14.8557870626006,
  'x': 1

In [22]:
replay_json['frames'][0]['planets']

{'0': {'current_production': 0,
  'docked_ships': [],
  'health': 2022,
  'id': 0,
  'owner': None,
  'remaining_production': 1141},
 '1': {'current_production': 0,
  'docked_ships': [],
  'health': 2022,
  'id': 1,
  'owner': None,
  'remaining_production': 1141},
 '10': {'current_production': 0,
  'docked_ships': [],
  'health': 1351,
  'id': 10,
  'owner': None,
  'remaining_production': 763},
 '11': {'current_production': 0,
  'docked_ships': [],
  'health': 1351,
  'id': 11,
  'owner': None,
  'remaining_production': 763},
 '2': {'current_production': 0,
  'docked_ships': [],
  'health': 2022,
  'id': 2,
  'owner': None,
  'remaining_production': 1141},
 '3': {'current_production': 0,
  'docked_ships': [],
  'health': 2022,
  'id': 3,
  'owner': None,
  'remaining_production': 1141},
 '4': {'current_production': 0,
  'docked_ships': [],
  'health': 3788,
  'id': 4,
  'owner': None,
  'remaining_production': 2139},
 '5': {'current_production': 0,
  'docked_ships': [],
  'health': 3

In [ ]:
replay_json.keys()

In [ ]:
replay_json['height']

In [ ]:
list(replay_json['frames'][0]['ships']['0']['0'])

In [ ]:
replay_json['num_frames']

In [ ]:
replay_json['planets'][0]

In [ ]:
for frame in replay_json['frames'][:1]:
    print(frame['ships']['1']['3'])
    print(frame['planets']['0'])

In [ ]:
for frame in replay_json['frames']:
    print(frame['events'])
replay_json['frames'][8]['events']

In [ ]:
replay_json['moves'][0]['0']

In [ ]:
planets = replay_json['num_players']
print(planets)

In [ ]:
max_ = -1000
sum_ = 0
count = 0
for i in tqdm(range(2000)):#replay_buffer.len):
    replay = replay_buffer.getNext()
    replay_json = getReplayContent(replay_dir + replay_buffer.getNext())
    if replay_json['num_players'] > 2:
        continue
    planets = replay_json['planets']
    no = len(planets)
    
    print(no)
    sum_ += no
    count += 1
    if(no > max_):
        max_ = no
print(max_)
print(sum_ / count)